In [1]:
import sasoptpy as so
import pandas as pd
import os
from math import sqrt
import random

In [2]:
w = 16
l = 12
df = pd.read_csv('../data/xt_pre_data.csv')
df['xbin'] = pd.cut(df.start_x, w, labels=False)
df['ybin'] = pd.cut(df.start_y, l, labels=False)
df['xbin_to'] = pd.cut(df.end_x, w, labels=False)
df['ybin_to'] = pd.cut(df.end_y, l, labels=False)

In [3]:
def get_matrix(values, xbins, ybins):
    indices = [(x,y) for x in range(xbins) for y in range(ybins)]
    x_cuts = values['xbin']
    y_cuts = values['ybin']
    bins = values.groupby([x_cuts, y_cuts]).size()
    return bins.reindex(indices, fill_value=0).unstack().fillna(0)

def get_transition_matrix(values, xbins, ybins):
    indices = [(x1,y1,x2,y2) for x1 in range(xbins) for y1 in range(ybins) for x2 in range(xbins) for y2 in range(ybins)]
    x_from = values['xbin']
    y_from = values['ybin']
    x_to = values['xbin_to']
    y_to = values['ybin_to']
    valid_occurences = values[values['result_name'] == 'success'].copy()
    occurence = values.groupby([x_from, y_from, x_to, y_to]).size()
    successful = valid_occurences.groupby([x_from, y_from, x_to, y_to]).size()
    occurence_2d = occurence.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    successful_2d = successful.reindex(indices).unstack([2,3], fill_value=0).fillna(0)
    # Only successful attempts?
    return successful_2d.div(occurence_2d.sum(axis=1), axis=0)

In [4]:
shots_df = get_matrix(df[df['type_name']=='shot'], w, l)
goals_df = get_matrix(df[(df['type_name']=='shot') & (df['result_name'] == 'success')], w, l)
moves_df = get_matrix(df[df['type_name'].isin(['dribble', 'pass', 'cross'])], w, l)
total_df = moves_df + shots_df

moves = moves_df / total_df
shots = shots_df / total_df
scores = (goals_df / shots_df).fillna(0)
T = get_transition_matrix(df[df['type_name'].isin(['dribble', 'pass', 'cross'])], w, l)

In [5]:
def base_model():
    model = so.Model(name='xThreatModel', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    model.add_constraints(
        (xT[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.set_objective(0, name='zero', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps solve solu solution.txt'
    # !{command}
    os.system(command)
    for v in model.get_variables():
        v.set_value(0)
    with open('solution.txt', 'r') as f:
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            model.get_variable(words[1]).set_value(float(words[2]))
    return model

In [6]:
m1 = base_model()

NOTE: Initialized model xThreatModel.


In [7]:
xT1 = m1.get_variable('xT')
so.get_solution_table(xT1)

,xT
"(0, 0)",0.001014
"(0, 1)",0.001490
"(0, 2)",0.001763
"(0, 3)",0.002306
"(0, 4)",0.002635
...,...
"(15, 7)",0.098388
"(15, 8)",0.029122
"(15, 9)",0.026838
"(15, 10)",0.025522


In [8]:
print(xT1[11,0].get_value(),xT1[11,7].get_value())

0.014957486 0.020520949


In [9]:
print(xT1[11,3].get_value(),xT1[11,4].get_value())

0.020823058 0.019536792


In [10]:
def symmetric_model():
    model = so.Model(name='xThreatModel_sym', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    err = model.add_variables(indices, name='error')
    err_abs = model.add_variables(indices, name='error_abs', lb=0)
    model.add_constraints(
        (xT[x,y] + err[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.add_constraints(
        (err_abs[x,y] >= err[x,y] for (x,y) in indices), name='abs_values1')
    model.add_constraints(
        (err_abs[x,y] >= -err[x,y] for (x,y) in indices), name='abs_values2')
    model.add_constraints(
        (xT[x,y] == xT[x, l-y-1] for (x,y) in indices), name='symm_con')
    model.add_constraint(so.expr_sum(err[x,y] for (x,y) in indices) == 0, name='zero_error_total')
    sum_err_abs = so.expr_sum(err_abs[x,y] for (x,y) in indices)
    model.set_objective(sum_err_abs, name='total_error', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps solve solu solution.txt'
    !{command}
    # os.system(command)
    for v in model.get_variables():
        v.set_value(0)
    with open('solution.txt', 'r') as f:
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            model.get_variable(words[1]).set_value(float(words[2]))
    return model

In [11]:
m2 = symmetric_model()

NOTE: Initialized model xThreatModel_sym.
Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 25 2020 

command line - cbc export.mps solve solu solution.txt (default strategy 1)
At line 1 NAME     xThreatModel_sym
At line 2 ROWS
At line 773 COLUMNS
At line 9535 RHS
At line 9551 RANGES
At line 9552 BOUNDS
At line 9937 ENDATA
Problem xThreatModel_sym has 769 rows, 576 columns and 17032 elements
Coin0008I xThreatModel_sym read with 0 errors
Presolve 577 (-192) rows, 480 (-96) columns and 11406 (-5626) elements
Perturbing problem by 0.001% of 67.409968 - largest nonzero change 0.00042036491 ( 0.00062359458%) - largest zero change 0
Optimal - objective value 0.14635766
After Postsolve, objective 0.14635766, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.1463576561 - 784 iterations time 0.052, Presolve 0.00
Total time (CPU seconds):       0.08   (Wallclock seconds):       0.08



In [12]:
xT2 = m2.get_variable('xT')
e2 = m2.get_variable('error')
so.get_solution_table(xT2)

,xT
"(0, 0)",0.001054
"(0, 1)",0.001548
"(0, 2)",0.002040
"(0, 3)",0.002368
"(0, 4)",0.002664
...,...
"(15, 7)",0.096434
"(15, 8)",0.039402
"(15, 9)",0.030494
"(15, 10)",0.025734


In [13]:
print(xT2[11,0].get_value(),xT2[11,7].get_value())

0.014837734 0.020427876


In [14]:
print(xT2[11,3].get_value(),xT2[11,4].get_value())

0.019836772 0.020427876


In [15]:
def dist(c1, c2):
    return sqrt((c1-(w-1))**2 + (c2-(l-1)/2)**2)

def sym_incremental_model():
    model = so.Model(name='xThreatModel_sym_inc', session=None)
    indices = [(x,y) for x in range(w) for y in range(l)]
    xT = model.add_variables(indices, name='xT')
    err = model.add_variables(indices, name='error')
    err_abs = model.add_variables(indices, name='error_abs', lb=0)
    model.add_constraints(
        (xT[x,y] + err[x,y] == shots.loc[x,y] * scores.loc[x,y] + moves.loc[x,y] * so.expr_sum(T.loc[(x,y),(z,w)] * xT[z,w] for (z,w) in indices) for (x,y) in indices), name='relation')
    model.add_constraints(
        (err_abs[x,y] >= err[x,y] for (x,y) in indices), name='abs_values1')
    model.add_constraints(
        (err_abs[x,y] >= -err[x,y] for (x,y) in indices), name='abs_values2')
    model.add_constraints(
        (xT[x,y] == xT[x, l-y-1] for (x,y) in indices), name='symm_con')
    model.add_constraint(so.expr_sum(err[x,y] for (x,y) in indices) == 0, name='zero_error_total')
    model.add_constraints(
        (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if dist(x,y) < dist(z,w) and x==z), name='same_row')
    model.add_constraints(
        (xT[x,y] >= xT[z, w] for (x,y) in indices for (z,w) in indices if dist(x,y) < dist(z,w) and y==w), name='same_col')
    sum_err_abs = so.expr_sum(err_abs[x,y] for (x,y) in indices)
    model.set_objective(sum_err_abs, name='total_error', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps presolve off solve solu solution.txt'
    !{command}
    #os.system(command)
    for v in model.get_variables():
        v.set_value(0)
    with open('solution.txt', 'r') as f:
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            model.get_variable(words[1]).set_value(float(words[2]))
    return model

In [16]:
m3 = sym_incremental_model()

NOTE: Initialized model xThreatModel_sym_inc.
Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 25 2020 

command line - cbc export.mps presolve off solve solu solution.txt (default strategy 1)
At line 1 NAME     xThreatModel_sym_inc
At line 2 ROWS
At line 3173 COLUMNS
At line 14325 RHS
At line 14341 RANGES
At line 14342 BOUNDS
At line 14727 ENDATA
Problem xThreatModel_sym_inc has 3169 rows, 576 columns and 21832 elements
Coin0008I xThreatModel_sym_inc read with 0 errors
Option for presolve changed from on to off
Perturbing problem by 0.001% of 68.680282 - largest nonzero change 0.00042734516 ( 0.00062222395%) - largest zero change 0
Optimal - objective value 0.15490441
Optimal objective 0.1549044117 - 2770 iterations time 0.362
Total time (CPU seconds):       0.40   (Wallclock seconds):       0.40



In [17]:
xT3 = m3.get_variable('xT')
e3 = m3.get_variable('error')
so.get_solution_table(xT3)
for y in range(l):
    print(y, xT3[11,y].get_value())

0 0.013798014
1 0.015327688
2 0.01666621
3 0.018234825
4 0.01826423
5 0.01826423
6 0.01826423
7 0.01826423
8 0.018234825
9 0.01666621
10 0.015327688
11 0.013798014


In [18]:
with pd.option_context('display.max_rows', None):
    print(so.get_solution_table(xT1, xT2, xT3))

                xT        xT        xT
(0, 0)    0.001014  0.001054  0.000969
(0, 1)    0.001490  0.001548  0.001424
(0, 2)    0.001763  0.002040  0.001876
(0, 3)    0.002306  0.002368  0.002176
(0, 4)    0.002635  0.002664  0.002453
(0, 5)    0.002820  0.002863  0.002633
(0, 6)    0.002849  0.002863  0.002633
(0, 7)    0.002617  0.002664  0.002453
(0, 8)    0.002218  0.002368  0.002176
(0, 9)    0.002071  0.002040  0.001876
(0, 10)   0.001530  0.001548  0.001424
(0, 11)   0.001029  0.001054  0.000969
(1, 0)    0.001516  0.001668  0.001537
(1, 1)    0.001929  0.002151  0.001979
(1, 2)    0.002535  0.002759  0.002537
(1, 3)    0.003046  0.003110  0.002863
(1, 4)    0.003250  0.003285  0.003020
(1, 5)    0.003448  0.003459  0.003183
(1, 6)    0.003396  0.003459  0.003183
(1, 7)    0.003206  0.003285  0.003020
(1, 8)    0.003132  0.003110  0.002863
(1, 9)    0.002810  0.002759  0.002537
(1, 10)   0.002191  0.002151  0.001979
(1, 11)   0.001708  0.001668  0.001537
(2, 0)    0.002118  0.002

In [19]:
table1 = so.get_solution_table(xT1)
table1.index = pd.MultiIndex.from_tuples(table1.index)
table1.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.001014  0.001516  0.002118  0.002609  0.003312  0.004090  0.005004   
   1   0.001490  0.001929  0.002515  0.003106  0.003830  0.004558  0.005738   
   2   0.001763  0.002535  0.003022  0.003550  0.004217  0.005050  0.006364   
   3   0.002306  0.003046  0.003533  0.003784  0.004627  0.005586  0.006593   
   4   0.002635  0.003250  0.003989  0.003988  0.004779  0.005739  0.006999   
   5   0.002820  0.003448  0.003856  0.004109  0.005046  0.006026  0.007403   
   6   0.002849  0.003396  0.003981  0.004312  0.005014  0.005933  0.007255   
   7   0.002617  0.003206  0.003554  0.004145  0.004955  0.005974  0.007194   
   8   0.002218  0.003132  0.003530  0.004042  0.004772  0.005721  0.007000   
   9   0.002071  0.002810  0.003280  0.003844  0.004418  0.005403  0.006612   
   10  0.001530  0.002191  0.002824  0.003387  0.004180  0.004915  0.006056   
   11  0.001029  0.001708  0.002113  0.002898  0.003562  0.004259  0.005323   

             7         8         9         10        11        12        13  \
xT 0   0.006172  0.007615  0.009165  0.011689  0.014957  0.017677  0.018394   
   1   0.007192  0.008621  0.010392  0.012981  0.016692  0.021262  0.024580   
   2   0.007522  0.009121  0.011123  0.014026  0.018087  0.024567  0.028702   
   3   0.008364  0.009596  0.011533  0.015221  0.020823  0.025055  0.035149   
   4   0.008476  0.010310  0.012054  0.015066  0.019537  0.029322  0.070550   
   5   0.008075  0.010009  0.012394  0.014949  0.019821  0.036462  0.077001   
   6   0.008413  0.010251  0.012648  0.015160  0.021144  0.036160  0.084869   
   7   0.008668  0.010072  0.011819  0.015216  0.020521  0.030363  0.068978   
   8   0.008439  0.009827  0.011759  0.015066  0.019864  0.025601  0.038418   
   9   0.008145  0.009449  0.011309  0.014427  0.018670  0.022498  0.026263   
   10  0.007473  0.008771  0.010548  0.013224  0.016773  0.020370  0.022982   
   11  0.006588  0.007905  0.009656  0.012247  0.015053  0.017643  0.019895   

             14        15  
xT 0   0.018630  0.018813  
   1   0.023152  0.025382  
   2   0.033817  0.030980  
   3   0.043171  0.039593  
   4   0.099819  0.091523  
   5   0.190227  0.382598  
   6   0.187113  0.428257  
   7   0.131806  0.098388  
   8   0.050468  0.029122  
   9   0.028164  0.026838  
   10  0.020692  0.025522  
   11  0.018745  0.023181

In [20]:
table2 = so.get_solution_table(xT2)
table2.index = pd.MultiIndex.from_tuples(table2.index)
table2.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.001054  0.001668  0.002189  0.002817  0.003443  0.004101  0.005111   
   1   0.001548  0.002151  0.002764  0.003293  0.004047  0.004732  0.005818   
   2   0.002040  0.002759  0.003220  0.003603  0.004287  0.005048  0.006319   
   3   0.002368  0.003110  0.003582  0.003816  0.004652  0.005559  0.006524   
   4   0.002664  0.003285  0.003982  0.004085  0.004863  0.005834  0.007007   
   5   0.002863  0.003459  0.003960  0.004279  0.004991  0.005934  0.007253   
   6   0.002863  0.003459  0.003960  0.004279  0.004991  0.005934  0.007253   
   7   0.002664  0.003285  0.003982  0.004085  0.004863  0.005834  0.007007   
   8   0.002368  0.003110  0.003582  0.003816  0.004652  0.005559  0.006524   
   9   0.002040  0.002759  0.003220  0.003603  0.004287  0.005048  0.006319   
   10  0.001548  0.002151  0.002764  0.003293  0.004047  0.004732  0.005818   
   11  0.001054  0.001668  0.002189  0.002817  0.003443  0.004101  0.005111   

             7         8         9         10        11        12        13  \
xT 0   0.006150  0.007569  0.009087  0.011584  0.014838  0.017763  0.018362   
   1   0.007139  0.008457  0.010268  0.012884  0.016533  0.020589  0.023646   
   2   0.007442  0.009017  0.011014  0.013905  0.017988  0.023043  0.028983   
   3   0.008193  0.009481  0.011560  0.014928  0.019837  0.025694  0.038413   
   4   0.008446  0.009870  0.011959  0.015099  0.020428  0.029827  0.071210   
   5   0.007927  0.009873  0.012579  0.015014  0.019775  0.036313  0.084849   
   6   0.007927  0.009873  0.012579  0.015014  0.019775  0.036313  0.084849   
   7   0.008446  0.009870  0.011959  0.015099  0.020428  0.029827  0.071210   
   8   0.008193  0.009481  0.011560  0.014928  0.019837  0.025694  0.038413   
   9   0.007442  0.009017  0.011014  0.013905  0.017988  0.023043  0.028983   
   10  0.007139  0.008457  0.010268  0.012884  0.016533  0.020589  0.023646   
   11  0.006150  0.007569  0.009087  0.011584  0.014838  0.017763  0.018362   

             14        15  
xT 0   0.018912  0.023150  
   1   0.023089  0.025734  
   2   0.033991  0.030494  
   3   0.049987  0.039402  
   4   0.100121  0.096434  
   5   0.190592  0.394944  
   6   0.190592  0.394944  
   7   0.100121  0.096434  
   8   0.049987  0.039402  
   9   0.033991  0.030494  
   10  0.023089  0.025734  
   11  0.018912  0.023150

In [21]:
table3 = so.get_solution_table(xT3)
table3.index = pd.MultiIndex.from_tuples(table3.index)
table3.unstack().transpose()

0         1         2         3         4         5         6   \
xT 0   0.000969  0.001537  0.002016  0.002592  0.003166  0.003771  0.004618   
   1   0.001424  0.001979  0.002543  0.003027  0.003723  0.004353  0.005365   
   2   0.001876  0.002537  0.002963  0.003312  0.003944  0.004647  0.005830   
   3   0.002176  0.002863  0.003292  0.003510  0.004279  0.005118  0.006014   
   4   0.002453  0.003020  0.003640  0.003755  0.004477  0.005382  0.006365   
   5   0.002633  0.003183  0.003640  0.003862  0.004601  0.005479  0.006723   
   6   0.002633  0.003183  0.003640  0.003862  0.004601  0.005479  0.006723   
   7   0.002453  0.003020  0.003640  0.003755  0.004477  0.005382  0.006365   
   8   0.002176  0.002863  0.003292  0.003510  0.004279  0.005118  0.006014   
   9   0.001876  0.002537  0.002963  0.003312  0.003944  0.004647  0.005830   
   10  0.001424  0.001979  0.002543  0.003027  0.003723  0.004353  0.005365   
   11  0.000969  0.001537  0.002016  0.002592  0.003166  0.003771  0.004618   

             7         8         9         10        11        12        13  \
xT 0   0.005670  0.006990  0.008406  0.010730  0.013798  0.016384  0.017240   
   1   0.006588  0.007812  0.009490  0.011922  0.015328  0.019140  0.022192   
   2   0.006876  0.008330  0.010189  0.012843  0.016666  0.021415  0.024795   
   3   0.007564  0.008757  0.010565  0.013764  0.018235  0.023565  0.033800   
   4   0.007625  0.009137  0.011028  0.013841  0.018264  0.027896  0.065061   
   5   0.007625  0.009339  0.011626  0.013841  0.018264  0.034470  0.075474   
   6   0.007625  0.009339  0.011626  0.013841  0.018264  0.034470  0.075474   
   7   0.007625  0.009137  0.011028  0.013841  0.018264  0.027896  0.065061   
   8   0.007564  0.008757  0.010565  0.013764  0.018235  0.023565  0.033800   
   9   0.006876  0.008330  0.010189  0.012843  0.016666  0.021415  0.024795   
   10  0.006588  0.007812  0.009490  0.011922  0.015328  0.019140  0.022192   
   11  0.005670  0.006990  0.008406  0.010730  0.013798  0.016384  0.017240   

             14        15  
xT 0   0.017945  0.022745  
   1   0.022192  0.025313  
   2   0.030232  0.030232  
   3   0.040207  0.040207  
   4   0.098406  0.098406  
   5   0.185911  0.429171  
   6   0.185911  0.429171  
   7   0.098406  0.098406  
   8   0.040207  0.040207  
   9   0.030232  0.030232  
   10  0.022192  0.025313  
   11  0.017945  0.022745

In [22]:
# Future Idea
def optimal_xT(xT_vals):
    model = so.Model('optimal_flow')
    indices = [(x,y) for x in range(w) for y in range(l)]
    nodes = ['source'] + indices + ['sink']
    
    use = model.add_variables(indices, name='use', vartype=so.binary)
    flow = model.add_variables(nodes, nodes, name='flow', vartype=so.binary)

    model.add_constraint(flow['source',7,5] == 1, name='initial_node')
    inflow = {(x,y): so.expr_sum(flow[i,j,x,y] for (i,j) in indices) + flow['source',x,y] for (x,y) in indices}
    outflow = {(x,y): so.expr_sum(flow[x,y,i,j] for (i,j) in indices) + flow[x,y,'sink'] for (x,y) in indices}
    model.add_constraints((inflow[x,y] == outflow[x,y] for (x,y) in indices), name='flow_balance')
    model.add_constraints((use[x,y] == inflow[x,y] for (x,y) in indices), name='use_if_flow')
    model.add_constraint(so.expr_sum(flow[x,y,'sink'] for (x,y) in indices) == 1, name='take_shot')
    model.add_constraint(so.expr_sum(use[x,y] for (x,y) in indices) == 6, name='limit_moves')
    model.add_constraints((flow[x,y,x,y] == 0 for (x,y) in indices), name='no_self_move')

    ## Replace this with probability matrix!!
    pass_ratings = so.expr_sum(T.loc[(x,y),(z,w)] * (xt_vals.iloc[z,w] - xt_vals.iloc[x,y]) * flow[x,y,z,w] for (x,y) in indices for (z,w) in indices)
    shot_rating = so.expr_sum(shots.loc[x,y] * scores.loc[x,y] * flow[x,y,'sink'] for (x,y) in indices)
    model.set_objective(-1* (shot_rating + pass_ratings), name='total_rating', sense='N')
    model.export_mps(filename='export.mps')
    command = 'cbc export.mps presolve off solve solu solution.txt'
    !{command}

xt_vals = table3.unstack()
optimal_xT(xt_vals)

NOTE: Initialized model optimal_flow.
Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 25 2020 

command line - cbc export.mps presolve off solve solu solution.txt (default strategy 1)
At line 1 NAME     optimal_flow
At line 2 ROWS
At line 583 COLUMNS
At line 75307 RHS
At line 75310 RANGES
At line 75311 BOUNDS
At line 113140 ENDATA
Problem optimal_flow has 579 rows, 37828 columns and 111553 elements
Coin0008I optimal_flow read with 0 errors
Option for presolve changed from on to off
Continuous objective value is -0.436492 - 0.07 seconds
Cgl0002I 192 variables fixed
Cgl0004I processed model has 385 rows, 37055 columns (37055 integer (37055 of which binary)) and 110591 elements
Coin3009W Conflict graph built in 0.306 seconds, density: 0.132%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0038I Initial state - 6 integers unsatisfied sum - 3
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. -0.431708 iterations 816
Cbc0038I Solution found of -0.431708
Cbc